In [47]:
import random
import itertools
import array
import sys
import pandas as pd
import matplotlib.pyplot as plt
import players
import ur
from multiprocessing import Process
from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
states = {'test':0.1}
qp1 = players.DeepQPlayer(0.5)

qp2 = players.QPlayer(0.5, states=states)
mxp = players.MaxPlayer(name='Max')
rnp = players.RandomPlayer(name='Random')
amp = players.AggressiveMaxPlayer(name='AMax')
rop = players.RosettaLoverPlayer(name='Rosetta')

m_res = []
r_res = []
am_res = []
rl_res = []
#qp_res = []

enemies = {'max':mxp,
           'rnd':rnp,
           'agg':amp,
           'ros':rop}
p_res = {'max':m_res,
         'rnd':r_res,
         'agg':am_res,
         'ros':rl_res}

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
ur.train(qp1, enemies, p_res, times=300)

ended learning 1 / 300
ended learning 16 / 300
ended learning 31 / 300
ended learning 46 / 300
ended learning 61 / 300
ended learning 76 / 300
ended learning 91 / 300
ended learning 106 / 300


In [53]:
ur.test_player(qp1, rnp, times=1000)

0.366

In [31]:
import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=23))
model.add(Dense(units=6, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [85]:
import numpy as np
x_train = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])
y_train = np.array([[1,0,1,1,1,1]])
model.fit(x_train, y_train, epochs=5, batch_size=32)
x_test = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])
y_test = np.array([[1,0,0,0,0,0]])
classes = model.predict(x_test, batch_size=128)
classes

Epoch 1/5
1/1 [==============================] - 0s 6ms/step - loss: 8.0670
Epoch 2/5
1/1 [==============================] - 0s 8ms/step - loss: 8.0665
Epoch 3/5
1/1 [==============================] - 0s 13ms/step - loss: 8.0660
Epoch 4/5
1/1 [==============================] - 0s 7ms/step - loss: 8.0655
Epoch 5/5
1/1 [==============================] - 0s 8ms/step - loss: 8.0651


array([[0.19983235, 0.00347814, 0.19968957, 0.19945303, 0.19911113,
        0.19843578]], dtype=float32)

0.9999999850988388

In [ ]:
data = {'max':m_res, 'rnd':r_res, 'agg':am_res, 'ros':rl_res, 'qp2':qp_res}
pd.DataFrame(data).plot()
plt.show()

In [ ]:
l = sorted([ (qp1.states[key], key) for key in qp1.states ])
l.reverse()
l

In [51]:
#human = HumanPlayer()
rp1 = players.RandomPlayer()
rp2 = players.RandomPlayer()
dqp = players.DeepQPlayer(0.05)
game = ur.Ur(qp1, rp2)
game.start_board()
#for i in range(100):
while not game.is_ended():
    game.player_move()
    print(game.get_state_list(qp1))

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, -1, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, -1, 1, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, -1, 1, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, -1, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0]
[0, -1, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0]
[0, -1, 0, 0, 0, 0, 0, -1, 0, 1, 0,